In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import Button, HBox, Label, Output, VBox

simulation_name = "cantilever"
import pyvista as pv
import yaml
from IPython.display import display


def plot_beam_2d(simulation_name):
    """Plot the beam deformation in 2D (XY-plane) over time steps."""

    # Load the result file
    simulation_path = Path.cwd() / simulation_name
    output_file = simulation_path / f"{simulation_name}-structure-beams.pvd"
    reader = pv.get_reader(output_file)

    # Load the element arc lengths
    with open(simulation_path / f"{simulation_name}_arc_length.yaml") as stream:
        arc_length_data = yaml.safe_load(stream)

    # Load the data for all time steps
    steps = reader.time_values
    grid_points = []
    cross_section_resultants_map = {
        "material_axial_force_GPs": "normal",
        "material_bending_moment_3_GPs": "bending",
        "material_shear_force_2_GPs": "shear",
    }
    cross_section_resultants_labels = {
        "normal": "Axial force",
        "bending": "Bending moment",
        "shear": "Shear force",
    }
    cell_data = {
        "normal": [],
        "bending": [],
        "shear": [],
    }
    for i_step, time in enumerate(steps):
        reader.set_active_time_point(i_step)
        mesh = reader.read()[0]
        grid_points.append(mesh.points.copy())
        for name in mesh.cell_data.keys():
            if name in cross_section_resultants_map:
                data_name = cross_section_resultants_map[name]
                cell_data[data_name].append(np.array(mesh.cell_data[name]))

    # Convert lists to numpy arrays for easier indexing
    grid_points = np.array(grid_points)
    for key in cell_data:
        data_from_pv = cell_data[key]
        data_time_steps = []
        for time_step in range(len(data_from_pv)):
            data_for_plot = []
            for i_element, value in enumerate(data_from_pv[time_step]):
                for i in range(2):
                    data_for_plot.append([arc_length_data[i_element][i], value])
                data_for_plot.append([np.nan, np.nan])
            data_time_steps.append(data_for_plot)
        cell_data[key] = np.array(data_time_steps)

    # Get the bounds for the displacement plot
    upper_bound = np.max(grid_points, axis=(0, 1))
    lower_bound = np.min(grid_points, axis=(0, 1))
    dimension = np.linalg.norm(upper_bound - lower_bound)

    # Check that the simulation is in plane
    if not np.isclose(upper_bound[2] - lower_bound[2], 0.0):
        raise ValueError(
            "The beam is not in the XY-plane. This plotting function only supports beams in the XY-plane."
        )

    #

    # -------------------------------------------------------
    # Output widget for the plot
    # -------------------------------------------------------
    out = Output()
    state = {"step": 0}

    def plot_step():
        step = state["step"]
        with out:
            out.clear_output(wait=True)
            fig = plt.figure(figsize=(10, 6))
            gs = fig.add_gridspec(3, 2, width_ratios=[3, 1])

            # First plot the beam deformation in XZ-plane
            ax0 = fig.add_subplot(gs[:, 0])
            ax0.plot(
                grid_points[state["step"], :, 0],
                grid_points[state["step"], :, 1],
                linewidth=3,
            )
            ax0.set_title(f"Beam configuration in space at time {steps[step]:.3f}")
            ax0.set_xlim(
                lower_bound[0] - dimension * 0.1, upper_bound[0] + dimension * 0.1
            )
            ax0.set_ylim(
                lower_bound[1] - dimension * 0.1, upper_bound[1] + dimension * 0.1
            )
            ax0.grid(True)
            ax0.set_xlabel("X")
            ax0.set_ylabel("Y")
            ax0.set_aspect("equal", "box")

            # Plot the cross-section resultants
            ax = [
                fig.add_subplot(gs[0, 1]),
                fig.add_subplot(gs[1, 1]),
                fig.add_subplot(gs[2, 1]),
            ]
            for i, (name, data) in enumerate(cell_data.items()):
                data = data[state["step"]]
                ax[i].plot(data[:, 0], data[:, 1])
                data_no_nan = data[~np.isnan(data[:, 1]), 1]
                y_bounds = [
                    np.min([0.0, np.min(data_no_nan)]),
                    np.max([0.0, np.max(data_no_nan)]),
                ]
                size = np.max([0.1, y_bounds[1] - y_bounds[0]])
                ax[i].set_ylim(y_bounds[0] - size * 0.1, y_bounds[1] + size * 0.1)
                ax[i].set_title(f"{cross_section_resultants_labels[name]} along beam")
                ax[i].grid(True)

            plt.tight_layout()
            plt.show()

    # -------------------------------------------------------
    # Step label
    # -------------------------------------------------------
    step_label = Label(f"Step: {state['step']}")

    def update_step_label():
        step_label.value = f"Step: {state['step']}"

    # -------------------------------------------------------
    # Buttons + logic
    # -------------------------------------------------------
    btn_first = Button(description="⏮ First")
    btn_prev = Button(description="◀ Prev")
    btn_next = Button(description="Next ▶")
    btn_last = Button(description="Last ⏭")

    def go_first(b):
        state["step"] = 0
        update_step_label()
        plot_step()

    def go_prev(b):
        if state["step"] > 0:
            state["step"] -= 1
        update_step_label()
        plot_step()

    def go_next(b):
        if state["step"] < len(steps) - 1:
            state["step"] += 1
        update_step_label()
        plot_step()

    def go_last(b):
        state["step"] = len(steps) - 1
        update_step_label()
        plot_step()

    btn_first.on_click(go_first)
    btn_prev.on_click(go_prev)
    btn_next.on_click(go_next)
    btn_last.on_click(go_last)

    # -------------------------------------------------------
    # Layout (buttons row, label row, then plot)
    # -------------------------------------------------------
    controls = HBox([btn_first, btn_prev, btn_next, btn_last])
    labels = HBox([step_label])

    ui = VBox([controls, out])

    # Show initial plot
    plot_step()
    display(ui)

# Beam finite element simulations with 4C

In this exercise, we will be creating, running and analyzing a beam finite element model, using the open source finite element software [4C](https://www.4c-multiphysics.org). 4C contains a variety of different finite element implementations for all kind of physical fields. In this exercise, we will exclusively be using reduced dimensional, geometrically exact beam elements, based on these two works:
- Jelenić, G., and Crisfield, M. A., 1999, “Geometrically Exact 3D Beam Theory: Implementation of a Strain-Invariant Finite Element for Statics and Dynamics,” Computer Methods in Applied Mechanics and Engineering, 171(1), pp. 141–171.
- Meier, C., 2016, “Geometrically Exact Finite Element Formulations for Slender Beams and Their Contact Interaction,” Dissertation, Technische Universität München.

For pre-processing, we will use the open source beam finite element generator [BeamMe](https://github.com/beamme-py/beamme). Post-processing is done directly within this notebook.

Note: The beam elements in 4C are developed for 3D analysis. However, in this exercise, we will only be considering plane examples.

In [ ]:
# We have to import the required packages, classes and functions.
import numpy as np
from beamme.core.mesh import Mesh
from beamme.four_c.element_beam import Beam3rLine2Line2
from beamme.four_c.material import MaterialReissner

from utils.lecture_utils import (
    create_beam_mesh_line_2d,
    create_boundary_condition_2d,
    run_four_c,
)

## Cantilever beam

In this first exercise, we will be creating the following cantilever beam model:

<img src="doc/cantilever.png" alt="Cantilever with force" width="400">

The beam has a length of $l = 0.3\text{m}$, a circular cross-section with width the radius $r = 0.01\text{m}$, a Young's modulus of $E = 2.1\cdot 10^{11}\text{N}/\text{mm}^2$. The left end of the beam is clamped, while a load of $F = 10000\text{N}$ is applied at the right end in negative $y$-direction.

We use two-noded linear beam finite elements based on the geometrically exact beam theory. The cantilever is discretized using 3 elements.

The following code block creates the beam finite element model using BeamMe and runs the simulation in 4C. At the end, the deformed shape of the beam and the cross-section force resultants are plotted. The simulation files are stored in a folder named `cantilever_1_0`.


In [ ]:
# In the beginning, we create the mesh container which will hold all elements,
# materials, and boundary conditions.
mesh = Mesh()

# Here we define the material properties for the beam elements.
# We use a Reissner beam material with specified radius, Young's modulus,
material = MaterialReissner(radius=0.01, youngs_modulus=2.1e11)

# Next, we create a straight beam connecting two points in space.
# For spatial discretization, we use 3 linear two-noded beam elements.
beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [0.3, 0], n_el=3
)

# To fix the cantilver beam, we apply Dirichlet boundary conditions to all
# positions and rotations at one node.
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y", "theta"]
)

# At the other end of the beam, we apply a Neumann boundary condition
# representing a downward force in y-direction.
create_boundary_condition_2d(
    mesh, beam_set["end"], bc_type="neumann", directions=["y"], values=[-10000.0]
)

# Now we can run the simulation. We have to give a unique name, in this case "cantilever_1_0",
# to store the results in a separate folder.
run_four_c(
    mesh=mesh,
    simulation_name="cantilever_1_0",
    n_steps=1,
    tol=1e-10,
)

# Display the solution (referenced by the unique name "cantilever_1_0").
plot_beam_2d("cantilever_1_0")

<div class="alert alert-info" role="alert">
  <strong>Exercise 2.1:</strong>

  We can see the deformed shape of the cantilever beam after applying the downward force. The cross-section force resultants are also plotted along the beam length. According to the linear Bernoulli-Euler beam theory, the bending moment should be linear, with the maximum value at the fixed end and zero at the free end. The shear force should be constant and the axial force should be zero. 
  
  1. The simulation results do not match these expectations. Can you explain why?
  1. What could be done to get the expected results? Adapt the code below accordingly and rerun the simulation.
</div>

In [ ]:
mesh = Mesh()
material = MaterialReissner(radius=0.01, youngs_modulus=2.1e11)
beam_set = create_beam_mesh_line_2d(
    mesh,
    Beam3rLine2Line2,
    material,
    [0, 0],
    [0.3, 0],
    n_el=100,  # We increase the number of elements here
)
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y", "theta"]
)
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["y"],
    values=[-1.0],  # We reduce the force here
)

run_four_c(
    mesh=mesh,
    simulation_name="cantilever_1_1",
    n_steps=1,
    tol=1e-10,
)

plot_beam_2d("cantilever_1_1")

<div class="alert alert-info" role="alert">
  <strong>Exercise 2.2:</strong>

  - We not want to change the load at the free end to an external moment with magnitude $M = 2000\text{Nm}$. Adapt the following code to achieve this (20 elements should be used for the discretization).

    *Hint:* Look at the function `create_boundary_condition_2d` and its arguments. The `directions` argument specifies which components of the boundary condition are applied, while the `values` argument specifies the corresponding magnitudes.

  - Increase the magnitude of the applied moment such that the beam exactly deforms into a half circle. 

    *Hint:* The moment of inertia for a circular cross-section is given as $I = \frac{\pi r^4}{4}$. The required moment to deform the beam into a half circle can then be calculated as $M = \frac{\pi}{2} \frac{EI}{l}$.

    *Hint:* This is highly non-linear problem. You might need to increase the number of load steps `n_steps` to get a converged solution. 
</div>

In [ ]:
youngs_modulus = 2.1e11
radius = 0.01
moment_of_inertia = np.pi * radius**4 / 4
length = 0.3

moment_magnitude = np.pi * youngs_modulus * moment_of_inertia / length

mesh = Mesh()
material = MaterialReissner(radius=radius, youngs_modulus=youngs_modulus)
beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [length, 0], n_el=20
)
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y", "theta"]
)
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["theta"],
    values=[moment_magnitude],
)

run_four_c(
    mesh=mesh,
    simulation_name="cantilever_1_2",
    n_steps=10,
    tol=1e-10,
)

plot_beam_2d("cantilever_1_2")

## Euler buckling column

In the first exercise, we will be creating the following cantilever beam model:

<img src="doc/cantilever.png" alt="Cantilever with force" width="400">

The beam has a length of $l = 0.3\text{m}$, a circular cross-section with width the radius $r = 0.01\text{m}$, a Young's modulus of $E = 2.1\cdot 10^{11}\text{N}/\text{mm}^2$. The left end of the beam is clamped, while a load of $F = 10000\text{N}$ is applied at the right end in negative $y$-direction.

We use two-noded linear beam finite elements based on the geometrically exact beam theory. The cantilever is discretized using 3 elements.

The following code block creates the beam finite element model using BeamMe and runs the simulation in 4C. At the end, the deformed shape of the beam and the cross-section force resultants are plotted. The simulation files are stored in a folder named `cantilever_1_0`.

In [ ]:
youngs_modulus = 2.1e11
radius = 0.01
moment_of_inertia = np.pi * radius**4 / 4
length = 0.3
critical_load = np.pi**2 * youngs_modulus * moment_of_inertia / (length**2)
print(critical_load)

# Create the mesh
mesh = Mesh()
material = MaterialReissner(radius=radius, youngs_modulus=youngs_modulus)
beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [length, 0], n_el=20
)

# Apply Dirichlet BCs at both ends
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y"]
)
create_boundary_condition_2d(
    mesh, beam_set["end"], bc_type="dirichlet", directions=["y"]
)

# Apply imperfection moments at both ends
imperfection_moment = 10
create_boundary_condition_2d(
    mesh,
    beam_set["start"],
    bc_type="neumann",
    directions=["theta"],
    values=[imperfection_moment],
    linear_increase=False,
)
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["theta"],
    values=[-imperfection_moment],
    linear_increase=False,
)

# Apply an axial force at the right end
force = 500000.0
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["x"],
    values=[-force],
    linear_increase=True,
)

run_four_c(
    mesh=mesh,
    simulation_name="cantilever_1_3",
    n_steps=100,
    tol=1e-10,
)

plot_beam_2d("cantilever_1_3")

# Plot the force displacement relationship
force, displacement = get_force_displacement_data("cantilever_1_3")
plt.plot(displacement[:, 1], force)
plt.axhline(y=critical_load, color="r", linestyle="--", label="Critical Load")
plt.xlabel("v")
plt.ylabel("F")
plt.title("Force-displacement plot")
plt.grid(True)
plt.show()

In [ ]:
from beamme.four_c.model_importer import import_four_c_model


def get_force_displacement_data(simulation_name):
    """Get the force and displacement data from the simulation results."""

    # Load the input file and extract the applied force.
    input_file_path = Path.cwd() / simulation_name / f"{simulation_name}.4C.yaml"
    input_file, _ = import_four_c_model(input_file_path)
    point_neumann_loads = input_file.fourc_input["DESIGN POINT NEUMANN CONDITIONS"]
    counter = 0
    for load in point_neumann_loads:
        if not np.all(np.array(load["FUNCT"]) == 0):
            force_value = np.linalg.norm(load["VAL"])
            # point_set = load["E"]
            counter += 1
    if counter != 1:
        raise ValueError(
            "Expected exactly one non-zero Neumann load in the input file."
        )

    # node_ids = [d["node_id"] for d in input_file.fourc_input["DNODE-NODE TOPOLOGY"] if d["d_id"] == point_set]
    # if len(node_ids) != 1:
    #     raise ValueError(f"Expected exactly one entry for d_id {point_set} in DNODE-NODE TOPOLOGY.")
    # node_id = node_ids[0]

    # coordinates = [d["COORD"] for d in input_file.fourc_input["NODE COORDS"] if d["id"] == node_id]
    # if len(coordinates) != 1:
    #     raise ValueError(f"Expected exactly one entry for node id {node_id} in NODE COORDS.")
    # coordinates = coordinates[0]

    # Load the result file
    simulation_path = Path.cwd() / simulation_name
    output_file = simulation_path / f"{simulation_name}-structure-beams.pvd"
    reader = pv.get_reader(output_file)

    # Load the data for all time steps
    steps = reader.time_values
    displacement = []
    force = []
    for i_step, time in enumerate(steps):
        reader.set_active_time_point(i_step)
        mesh = reader.read()[0]
        reference_coordinates = mesh.points - mesh.point_data["displacement"]
        difference = reference_coordinates - [0.15, 0.0, 0.0]
        distances = np.linalg.norm(difference, axis=1)
        closest_point_index = np.argmin(distances)
        displacement.append(mesh.point_data["displacement"][closest_point_index][:2])
        force.append(force_value * time)

    return np.array(force), np.array(displacement)